In [1]:
!pip install Kneed

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.cluster.hierarchy as sch
import statsmodels.formula.api as smf
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from kneed import KneeLocator
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Business Problem
Creating clusters on Global Development Measurement dataset.
Data Set Details:
###The dataset has information about important economic and development metrics related to various countries across the globe.

In [3]:
wdm_data = pd.read_excel("World_development_mesurement.xlsx")
wdm_data

,Birth Rate,Business Tax Rate,CO2 Emissions,Country,Days to Start Business,Ease of Business,Energy Usage,GDP,Health Exp % GDP,Health Exp/Capita,...,Life Expectancy Male,Mobile Phone Usage,Number of Records,Population 0-14,Population 15-64,Population 65+,Population Total,Population Urban,Tourism Inbound,Tourism Outbound
0,0.020,NaN,87931.0,Algeria,NaN,NaN,26998.0,"$54,790,058,957",0.035,$60,...,67.0,0.0,1,0.342,0.619,0.039,31719449,0.599,"$102,000,000","$193,000,000"
1,0.050,NaN,9542.0,Angola,NaN,NaN,7499.0,"$9,129,594,819",0.034,$22,...,44.0,0.0,1,0.476,0.499,0.025,13924930,0.324,"$34,000,000","$146,000,000"
2,0.043,NaN,1617.0,Benin,NaN,NaN,1983.0,"$2,359,122,303",0.043,$15,...,53.0,0.0,1,0.454,0.517,0.029,6949366,0.383,"$77,000,000","$50,000,000"
3,0.027,NaN,4276.0,Botswana,NaN,NaN,1836.0,"$5,788,311,645",0.047,$152,...,49.0,0.1,1,0.383,0.587,0.029,1755375,0.532,"$227,000,000","$209,000,000"
4,0.046,NaN,1041.0,Burkina Faso,NaN,NaN,NaN,"$2,610,959,139",0.051,$12,...,49.0,0.0,1,0.468,0.505,0.028,11607944,0.178,"$23,000,000","$30,000,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,NaN,NaN,NaN,Turks and Caicos Islands,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,32427,0.911,NaN,NaN
2700,0.013,46.4%,NaN,United States,5.0,4.0,2132446.0,"$16,244,600,000,000",0.179,"$8,895",...,76.0,1.0,1,0.196,0.667,0.136,313873685,0.811,"$200,092,000,000","$126,573,000,000"
2701,0.015,41.9%,NaN,Uruguay,7.0,85.0,NaN,"$50,004,354,667",0.089,"$1,308",...,74.0,1.5,1,0.220,0.639,0.140,3395253,0.948,"$2,222,000,000","$1,028,000,000"
2702,0.020,61.9%,NaN,"Venezuela, RB",144.0,180.0,NaN,"$381,286,223,859",0.046,$593,...,72.0,1.0,1,0.288,0.652,0.060,29954782,0.889,"$904,000,000","$3,202,000,000"


# Exploratory Data Analysis

In [4]:
wdm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2704 entries, 0 to 2703
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Birth Rate              2585 non-null   float64
 1   Business Tax Rate       1423 non-null   object 
 2   CO2 Emissions           2125 non-null   float64
 3   Country                 2704 non-null   object 
 4   Days to Start Business  1718 non-null   float64
 5   Ease of Business        185 non-null    float64
 6   Energy Usage            1785 non-null   float64
 7   GDP                     2494 non-null   object 
 8   Health Exp % GDP        2395 non-null   float64
 9   Health Exp/Capita       2395 non-null   object 
 10  Hours to do Tax         1416 non-null   float64
 11  Infant Mortality Rate   2444 non-null   float64
 12  Internet Usage          2531 non-null   float64
 13  Lending Interest        1880 non-null   float64
 14  Life Expectancy Female  2568 non-null   

In [5]:
wdm_data.shape

(2704, 25)

In [6]:
wdm_data.columns

Index(['Birth Rate', 'Business Tax Rate', 'CO2 Emissions', 'Country',
       'Days to Start Business', 'Ease of Business', 'Energy Usage', 'GDP',
       'Health Exp % GDP', 'Health Exp/Capita', 'Hours to do Tax',
       'Infant Mortality Rate', 'Internet Usage', 'Lending Interest',
       'Life Expectancy Female', 'Life Expectancy Male', 'Mobile Phone Usage',
       'Number of Records', 'Population 0-14', 'Population 15-64',
       'Population 65+', 'Population Total', 'Population Urban',
       'Tourism Inbound', 'Tourism Outbound'],
      dtype='object')

In [7]:
wdm_data.isnull().sum()

Birth Rate                 119
Business Tax Rate         1281
CO2 Emissions              579
Country                      0
Days to Start Business     986
Ease of Business          2519
Energy Usage               919
GDP                        210
Health Exp % GDP           309
Health Exp/Capita          309
Hours to do Tax           1288
Infant Mortality Rate      260
Internet Usage             173
Lending Interest           824
Life Expectancy Female     136
Life Expectancy Male       136
Mobile Phone Usage         167
Number of Records            0
Population 0-14            220
Population 15-64           220
Population 65+             220
Population Total             0
Population Urban            26
Tourism Inbound            368
Tourism Outbound           471
dtype: int64

In [8]:
wdm_data.describe()

,Birth Rate,CO2 Emissions,Days to Start Business,Ease of Business,Energy Usage,Health Exp % GDP,Hours to do Tax,Infant Mortality Rate,Internet Usage,Lending Interest,Life Expectancy Female,Life Expectancy Male,Mobile Phone Usage,Number of Records,Population 0-14,Population 15-64,Population 65+,Population Total,Population Urban
count,2585.000000,2.125000e+03,1718.000000,185.000000,1.785000e+03,2395.000000,1416.000000,2444.000000,2531.000000,1880.000000,2568.000000,2568.000000,2537.000000,2704.0,2484.000000,2484.000000,2484.000000,2.704000e+03,2678.000000
mean,0.022715,1.423071e+05,39.999418,94.875676,7.723684e+04,0.064170,302.068503,0.032813,0.239747,0.148103,71.159268,66.461838,0.578124,1.0,0.303051,0.623481,0.073465,3.145729e+07,0.562953
std,0.011338,6.159288e+05,57.318588,54.791906,2.599239e+05,0.026325,273.299686,0.029968,0.260132,0.171829,10.708262,9.390217,0.474647,0.0,0.106577,0.069660,0.049541,1.242894e+08,0.245897
min,0.007000,7.000000e+00,1.000000,1.000000,8.000000e+00,0.008000,12.000000,0.002000,0.000000,0.005000,39.000000,37.000000,0.000000,1.0,0.118000,0.474000,0.003000,1.887600e+04,0.082000
25%,0.013000,1.360000e+03,13.000000,48.000000,3.737000e+03,0.046000,156.000000,0.009000,0.000000,0.080000,64.000000,61.000000,0.100000,1.0,0.204750,0.559000,0.033000,8.783360e+05,0.356000
50%,0.020000,8.529000e+03,26.000000,94.000000,1.433800e+04,0.061000,239.000000,0.020000,0.100000,0.120000,75.000000,69.000000,0.500000,1.0,0.299500,0.642500,0.052000,5.800324e+06,0.560000
75%,0.031000,5.910800e+04,45.000000,142.000000,4.185200e+04,0.079000,344.750000,0.053000,0.400000,0.173000,79.000000,73.250000,0.900000,1.0,0.402000,0.676000,0.112000,2.014089e+07,0.765000
max,0.053000,8.286892e+06,694.000000,189.000000,2.727728e+06,0.225000,2600.000000,0.141000,1.000000,4.965000,87.000000,88.000000,2.900000,1.0,0.500000,0.858000,0.244000,1.350695e+09,1.000000


In [9]:
wdm_data[wdm_data.duplicated()].shape

(0, 25)

In [10]:
wdm_data['Business Tax Rate'] = wdm_data['Business Tax Rate'].str.rstrip('%').astype(float)

wdm_data['GDP'] = wdm_data['GDP'].str.replace('$','').str.replace(',','')
wdm_data['GDP'] = pd.to_numeric(wdm_data['GDP'],errors='coerce')
wdm_data['GDP'] = wdm_data['GDP'].fillna(0).astype(int)

wdm_data['Health Exp/Capita'] = wdm_data['Health Exp/Capita'].str.rstrip('$').str.replace('$','').str.replace(',','')
wdm_data['Health Exp/Capita'] = wdm_data['Health Exp/Capita'].astype(float)

wdm_data['Tourism Inbound'] = wdm_data['Tourism Inbound'].str.rstrip('$').str.replace('$','').str.replace(',','')
wdm_data['Tourism Inbound'] = wdm_data['Tourism Inbound'].astype(float)

wdm_data['Tourism Outbound'] = wdm_data['Tourism Outbound'].str.rstrip('$').str.replace('$','').str.replace(',','')
wdm_data['Tourism Outbound'] = wdm_data['Tourism Outbound'].astype(float)

In [11]:
wdm_data

,Birth Rate,Business Tax Rate,CO2 Emissions,Country,Days to Start Business,Ease of Business,Energy Usage,GDP,Health Exp % GDP,Health Exp/Capita,...,Life Expectancy Male,Mobile Phone Usage,Number of Records,Population 0-14,Population 15-64,Population 65+,Population Total,Population Urban,Tourism Inbound,Tourism Outbound
0,0.020,NaN,87931.0,Algeria,NaN,NaN,26998.0,-2147483648,0.035,60.0,...,67.0,0.0,1,0.342,0.619,0.039,31719449,0.599,1.020000e+08,1.930000e+08
1,0.050,NaN,9542.0,Angola,NaN,NaN,7499.0,-2147483648,0.034,22.0,...,44.0,0.0,1,0.476,0.499,0.025,13924930,0.324,3.400000e+07,1.460000e+08
2,0.043,NaN,1617.0,Benin,NaN,NaN,1983.0,-2147483648,0.043,15.0,...,53.0,0.0,1,0.454,0.517,0.029,6949366,0.383,7.700000e+07,5.000000e+07
3,0.027,NaN,4276.0,Botswana,NaN,NaN,1836.0,-2147483648,0.047,152.0,...,49.0,0.1,1,0.383,0.587,0.029,1755375,0.532,2.270000e+08,2.090000e+08
4,0.046,NaN,1041.0,Burkina Faso,NaN,NaN,NaN,-2147483648,0.051,12.0,...,49.0,0.0,1,0.468,0.505,0.028,11607944,0.178,2.300000e+07,3.000000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,NaN,NaN,NaN,Turks and Caicos Islands,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,32427,0.911,NaN,NaN
2700,0.013,46.4,NaN,United States,5.0,4.0,2132446.0,-2147483648,0.179,8895.0,...,76.0,1.0,1,0.196,0.667,0.136,313873685,0.811,2.000920e+11,1.265730e+11
2701,0.015,41.9,NaN,Uruguay,7.0,85.0,NaN,-2147483648,0.089,1308.0,...,74.0,1.5,1,0.220,0.639,0.140,3395253,0.948,2.222000e+09,1.028000e+09
2702,0.020,61.9,NaN,"Venezuela, RB",144.0,180.0,NaN,-2147483648,0.046,593.0,...,72.0,1.0,1,0.288,0.652,0.060,29954782,0.889,9.040000e+08,3.202000e+09


In [12]:
wdm_data.Country.nunique()

208

In [13]:
wdm_data.dtypes

Birth Rate                float64
Business Tax Rate         float64
CO2 Emissions             float64
Country                    object
Days to Start Business    float64
Ease of Business          float64
Energy Usage              float64
GDP                         int32
Health Exp % GDP          float64
Health Exp/Capita         float64
Hours to do Tax           float64
Infant Mortality Rate     float64
Internet Usage            float64
Lending Interest          float64
Life Expectancy Female    float64
Life Expectancy Male      float64
Mobile Phone Usage        float64
Number of Records           int64
Population 0-14           float64
Population 15-64          float64
Population 65+            float64
Population Total            int64
Population Urban          float64
Tourism Inbound           float64
Tourism Outbound          float64
dtype: object

# Feature Engineering

In [14]:
wdm_data.rename(columns = {'Birth Rate': 'Birth_Rate',
                           'Business Tax Rate': 'Business_Tax_Rate',
                           'CO2 Emissions': 'CO2_Emissions',
                           'Days to Start Business': 'Days_to_Start_Business',
                           'Ease of Business': 'Ease_of_Business',
                           'Energy Usage': 'Energy_Usage',
                           'Health Exp % GDP': 'Health_Exp%_GDP',
                           'Health Exp/Capita': 'Health_Exp/Capita',
                           'Hours to do Tax': 'Hours_to_do_Tax',
                           'Infant Mortality Rate': 'Infant_Mortality_Rate',
                           'Internet Usage': 'Internet_Usage',
                           'Lending Interest': 'Lending_Interest',
                           'Life Expectancy Female': 'Life_Expectancy_Female',
                           'Life Expectancy Male': 'Life_Expectancy_Male',
                           'Mobile Phone Usage': 'Mobile_Phone_Usage',
                           'Number of Records': 'Number_of_Records',
                           'Population 0-14': 'Population_0-14',
                           'Population 15-64': 'Population_15-64',
                           'Population 65+': 'Population_65+',
                           'Population Total': 'Population_Total',
                           'Population Urban': 'Population_Urban',
                           'Tourism Inbound': 'Tourism_Inbound',
                           'Tourism Outbound': 'Tourism_Outbound'}, inplace=True)

In [15]:
wdm_data.head()

,Birth_Rate,Business_Tax_Rate,CO2_Emissions,Country,Days_to_Start_Business,Ease_of_Business,Energy_Usage,GDP,Health_Exp%_GDP,Health_Exp/Capita,...,Life_Expectancy_Male,Mobile_Phone_Usage,Number_of_Records,Population_0-14,Population_15-64,Population_65+,Population_Total,Population_Urban,Tourism_Inbound,Tourism_Outbound
0,0.020,NaN,87931.0,Algeria,NaN,NaN,26998.0,-2147483648,0.035,60.0,...,67.0,0.0,1,0.342,0.619,0.039,31719449,0.599,102000000.0,193000000.0
1,0.050,NaN,9542.0,Angola,NaN,NaN,7499.0,-2147483648,0.034,22.0,...,44.0,0.0,1,0.476,0.499,0.025,13924930,0.324,34000000.0,146000000.0
2,0.043,NaN,1617.0,Benin,NaN,NaN,1983.0,-2147483648,0.043,15.0,...,53.0,0.0,1,0.454,0.517,0.029,6949366,0.383,77000000.0,50000000.0
3,0.027,NaN,4276.0,Botswana,NaN,NaN,1836.0,-2147483648,0.047,152.0,...,49.0,0.1,1,0.383,0.587,0.029,1755375,0.532,227000000.0,209000000.0
4,0.046,NaN,1041.0,Burkina Faso,NaN,NaN,NaN,-2147483648,0.051,12.0,...,49.0,0.0,1,0.468,0.505,0.028,11607944,0.178,23000000.0,30000000.0


In [16]:
data2 = wdm_data.drop(columns = ["Ease_of_Business","Number_of_Records"], axis=1)

In [17]:
data2.head()

,Birth_Rate,Business_Tax_Rate,CO2_Emissions,Country,Days_to_Start_Business,Energy_Usage,GDP,Health_Exp%_GDP,Health_Exp/Capita,Hours_to_do_Tax,...,Life_Expectancy_Female,Life_Expectancy_Male,Mobile_Phone_Usage,Population_0-14,Population_15-64,Population_65+,Population_Total,Population_Urban,Tourism_Inbound,Tourism_Outbound
0,0.020,NaN,87931.0,Algeria,NaN,26998.0,-2147483648,0.035,60.0,NaN,...,71.0,67.0,0.0,0.342,0.619,0.039,31719449,0.599,102000000.0,193000000.0
1,0.050,NaN,9542.0,Angola,NaN,7499.0,-2147483648,0.034,22.0,NaN,...,47.0,44.0,0.0,0.476,0.499,0.025,13924930,0.324,34000000.0,146000000.0
2,0.043,NaN,1617.0,Benin,NaN,1983.0,-2147483648,0.043,15.0,NaN,...,57.0,53.0,0.0,0.454,0.517,0.029,6949366,0.383,77000000.0,50000000.0
3,0.027,NaN,4276.0,Botswana,NaN,1836.0,-2147483648,0.047,152.0,NaN,...,52.0,49.0,0.1,0.383,0.587,0.029,1755375,0.532,227000000.0,209000000.0
4,0.046,NaN,1041.0,Burkina Faso,NaN,NaN,-2147483648,0.051,12.0,NaN,...,52.0,49.0,0.0,0.468,0.505,0.028,11607944,0.178,23000000.0,30000000.0


In [18]:
data2.isnull().sum()

Birth_Rate                 119
Business_Tax_Rate         1281
CO2_Emissions              579
Country                      0
Days_to_Start_Business     986
Energy_Usage               919
GDP                          0
Health_Exp%_GDP            309
Health_Exp/Capita          309
Hours_to_do_Tax           1288
Infant_Mortality_Rate      260
Internet_Usage             173
Lending_Interest           824
Life_Expectancy_Female     136
Life_Expectancy_Male       136
Mobile_Phone_Usage         167
Population_0-14            220
Population_15-64           220
Population_65+             220
Population_Total             0
Population_Urban            26
Tourism_Inbound            368
Tourism_Outbound           471
dtype: int64

In [19]:
data2.dtypes

Birth_Rate                float64
Business_Tax_Rate         float64
CO2_Emissions             float64
Country                    object
Days_to_Start_Business    float64
Energy_Usage              float64
GDP                         int32
Health_Exp%_GDP           float64
Health_Exp/Capita         float64
Hours_to_do_Tax           float64
Infant_Mortality_Rate     float64
Internet_Usage            float64
Lending_Interest          float64
Life_Expectancy_Female    float64
Life_Expectancy_Male      float64
Mobile_Phone_Usage        float64
Population_0-14           float64
Population_15-64          float64
Population_65+            float64
Population_Total            int64
Population_Urban          float64
Tourism_Inbound           float64
Tourism_Outbound          float64
dtype: object

In [20]:
skew = data2.skew().sort_values(ascending=False)
kurtosis = data2.kurt().sort_values(ascending=False)

In [21]:
print("Skewness:\n",skew, "\n\n", "Kurtosis:\n",kurtosis)

Skewness:
 Lending_Interest          15.536119
Population_Total           8.788164
CO2_Emissions              8.654039
Days_to_Start_Business     7.437592
Tourism_Inbound            7.114871
Energy_Usage               7.062230
Tourism_Outbound           5.210304
Hours_to_do_Tax            4.491308
Business_Tax_Rate          4.342568
Health_Exp/Capita          2.817482
GDP                        1.362504
Health_Exp%_GDP            1.179757
Infant_Mortality_Rate      1.049118
Internet_Usage             0.940966
Population_65+             0.919841
Birth_Rate                 0.662151
Mobile_Phone_Usage         0.569605
Population_0-14            0.059660
Population_Urban           0.012519
Population_15-64          -0.186881
Life_Expectancy_Male      -0.705485
Life_Expectancy_Female    -0.891556
dtype: float64 

 Kurtosis:
 Lending_Interest          374.122539
CO2_Emissions              83.080721
Population_Total           82.383028
Days_to_Start_Business     77.661930
Tourism_Inbound     

In [22]:
data2.describe().T

,count,mean,std,min,25%,50%,75%,max
Birth_Rate,2585.0,2.271489e-02,1.133843e-02,7.000000e-03,1.300000e-02,2.000000e-02,3.100000e-02,5.300000e-02
Business_Tax_Rate,1423.0,4.896690e+01,4.095412e+01,8.200000e+00,3.240000e+01,4.100000e+01,5.155000e+01,3.391000e+02
CO2_Emissions,2125.0,1.423071e+05,6.159288e+05,7.000000e+00,1.360000e+03,8.529000e+03,5.910800e+04,8.286892e+06
Days_to_Start_Business,1718.0,3.999942e+01,5.731859e+01,1.000000e+00,1.300000e+01,2.600000e+01,4.500000e+01,6.940000e+02
Energy_Usage,1785.0,7.723684e+04,2.599239e+05,8.000000e+00,3.737000e+03,1.433800e+04,4.185200e+04,2.727728e+06
GDP,2704.0,-1.440043e+09,1.252856e+09,-2.147484e+09,-2.147484e+09,-2.147484e+09,0.000000e+00,2.134105e+09
Health_Exp%_GDP,2395.0,6.416952e-02,2.632460e-02,8.000000e-03,4.600000e-02,6.100000e-02,7.900000e-02,2.250000e-01
Health_Exp/Capita,2395.0,8.013971e+02,1.473448e+03,2.000000e+00,5.000000e+01,1.930000e+02,6.650000e+02,9.908000e+03
Hours_to_do_Tax,1416.0,3.020685e+02,2.732997e+02,1.200000e+01,1.560000e+02,2.390000e+02,3.447500e+02,2.600000e+03
Infant_Mortality_Rate,2444.0,3.281301e-02,2.996760e-02,2.000000e-03,9.000000e-03,2.000000e-02,5.300000e-02,1.410000e-01


In [23]:
data2.corr()

,Birth_Rate,Business_Tax_Rate,CO2_Emissions,Days_to_Start_Business,Energy_Usage,GDP,Health_Exp%_GDP,Health_Exp/Capita,Hours_to_do_Tax,Infant_Mortality_Rate,...,Life_Expectancy_Female,Life_Expectancy_Male,Mobile_Phone_Usage,Population_0-14,Population_15-64,Population_65+,Population_Total,Population_Urban,Tourism_Inbound,Tourism_Outbound
Birth_Rate,1.000000,0.288982,-0.175750,0.169013,-0.169664,0.196894,-0.227653,-0.463907,0.069324,0.872535,...,-0.868174,-0.825745,-0.558411,0.955966,-0.912837,-0.773014,-0.057851,-0.606933,-0.287021,-0.296049
Business_Tax_Rate,0.288982,1.000000,0.022298,0.013339,0.059210,0.190359,0.034727,-0.110941,0.138647,0.341270,...,-0.284273,-0.312970,-0.233985,0.218907,-0.258508,-0.106498,0.058711,-0.123177,-0.017257,-0.021214
CO2_Emissions,-0.175750,0.022298,1.000000,-0.055666,0.988715,-0.120333,0.163236,0.254039,0.081485,-0.135421,...,0.135219,0.141452,0.054400,-0.198328,0.190808,0.160403,0.721145,0.098051,0.696357,0.650080
Days_to_Start_Business,0.169013,0.013339,-0.055666,1.000000,-0.069418,0.038452,-0.147686,-0.181849,0.148423,0.200082,...,-0.197764,-0.213764,-0.199868,0.203978,-0.166224,-0.201973,0.001357,-0.092493,-0.122206,-0.117471
Energy_Usage,-0.169664,0.059210,0.988715,-0.069418,1.000000,-0.089589,0.253146,0.278551,0.074673,-0.120019,...,0.125224,0.132984,0.045198,-0.184007,0.171528,0.148971,0.677971,0.094218,0.742379,0.703488
GDP,0.196894,0.190359,-0.120333,0.038452,-0.089589,1.000000,-0.023956,-0.174119,-0.151855,0.213081,...,-0.183555,-0.181346,-0.189102,0.276602,-0.247965,-0.246594,-0.130172,-0.208023,-0.171963,-0.169204
Health_Exp%_GDP,-0.227653,0.034727,0.163236,-0.147686,0.253146,-0.023956,1.000000,0.444986,-0.079755,-0.165117,...,0.208335,0.219441,0.187336,-0.292266,0.093604,0.489835,-0.054097,0.205994,0.379051,0.371141
Health_Exp/Capita,-0.463907,-0.110941,0.254039,-0.181849,0.278551,-0.174119,0.444986,1.000000,-0.213454,-0.452413,...,0.499595,0.540419,0.460803,-0.529232,0.362444,0.623775,-0.005934,0.520300,0.556093,0.579977
Hours_to_do_Tax,0.069324,0.138647,0.081485,0.148423,0.074673,-0.151855,-0.079755,-0.213454,1.000000,0.133808,...,-0.104772,-0.154317,-0.103461,0.075677,-0.084376,-0.043624,0.146920,0.010022,-0.058792,-0.040801
Infant_Mortality_Rate,0.872535,0.341270,-0.135421,0.200082,-0.120019,0.213081,-0.165117,-0.452413,0.133808,1.000000,...,-0.933001,-0.912240,-0.599383,0.822479,-0.798960,-0.649284,0.006897,-0.619742,-0.262693,-0.267785
